## Berechnungen

In diesem Notebook ist es möglich den Mittelwert, Varianz und Standardabeichung für Messwerte zu berechnen.   
Zudem wird die Gieränderung in rad bestimmt. Die Ergebnisse können zurück in die Merkmal CSV Datei geschrieben werden.

In [3]:
import pandas
import numpy as np
import numpy.testing as npt
import math
import os

In [4]:
#os.remove("merkmaleRoh.csv")

In [5]:
featuresDf = pandas.read_csv("merkmaleRoh.csv")
featuresDf.describe()

,Zeitstempel,Breitengrad,Laengengrad,Geschwindigkeit,Messwerte,StartBewegungsD,StartBelichtung,Belichtungszeit
count,4.033000e+03,4033.000000,4033.000000,4033.000000,4033.0,4.033000e+03,4.033000e+03,4.033000e+03
mean,1.520432e+12,53.614175,10.163583,13.797908,10.0,2.767638e+14,2.767639e+14,2.457690e+07
std,3.118401e+05,0.001194,0.014844,5.423002,0.0,3.118390e+11,3.118374e+11,5.764559e+06
min,1.520432e+12,53.610508,10.137785,0.000000,10.0,2.762160e+14,2.762161e+14,8.963045e+06
25%,1.520432e+12,53.613804,10.151080,11.520000,10.0,2.765011e+14,2.765012e+14,2.000368e+07
50%,1.520432e+12,53.614390,10.161076,15.264000,10.0,2.767615e+14,2.767615e+14,2.000368e+07
75%,1.520432e+12,53.615047,10.179163,17.352000,10.0,2.770310e+14,2.770311e+14,2.999632e+07
max,1.520433e+12,53.615852,10.185279,23.544000,10.0,2.773140e+14,2.773141e+14,2.999632e+07


In [6]:
featuresDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4033 entries, 0 to 4032
Data columns (total 15 columns):
Zeitstempel          4033 non-null int64
Breitengrad          4033 non-null float64
Laengengrad          4033 non-null float64
Geschwindigkeit      4033 non-null float64
AccelerometerX       4033 non-null object
AccelerometerY       4033 non-null object
AccelerometerZ       4033 non-null object
Azimuth              4033 non-null object
Nick                 4033 non-null object
Roll                 4033 non-null object
SensorZeitstempel    4033 non-null object
Messwerte            4033 non-null int64
StartBewegungsD      4033 non-null int64
StartBelichtung      4033 non-null int64
Belichtungszeit      4033 non-null int64
dtypes: float64(3), int64(5), object(7)
memory usage: 472.7+ KB


In [7]:
# Zeigt die ersten 5 Reihen Beschleunigungssensordaten der X-Achse 
featuresDf.AccelerometerX.head()

0    0.04156 0.00258 -0.01328 -0.0413 -0.03307 -0.0...
1    0.08288 0.02032 -0.06039 -0.07898 0.02872 0.02...
2    -0.01467 -0.02708 -0.05233 -0.02657 -0.05193 -...
3    -0.05236 -0.08788 -0.02436 -0.03483 0.00275 0....
4    -0.10448 -0.05297 -0.0424 -0.01863 -0.01493 0....
Name: AccelerometerX, dtype: object

Die folgenden Spalten des Datenframes haben als Type ein Python object.   
AccelerometerX       non-null object   
AccelerometerY       non-null object   
AccelerometerZ       non-null object   
Azimuth              non-null object   
Nick                 non-null object   
Roll                 non-null object   
Implizit haben diese Spalten als Typ Strings. Aber in einem DataFrame werden Strings als    
Python objects erkannt. Um den Datentyp auf float zu casten wird aus den Spalten zunächst ein Liste erstellt.    
Welche dann wiederum als Numpy Array auf den Typ float konvertiert wird.   

In [8]:
accXList = featuresDf.AccelerometerX.str.split(" ").tolist()

In [9]:
# Direkt mit dem Numpy Array ist es nicht möglich die Konvertierung durchzuführen, weil die Arrays von Typ object sind.   
pandas.DataFrame(featuresDf.AccelerometerX.str.split(" ")).values[1]

array([ ['0.08288', '0.02032', '-0.06039', '-0.07898', '0.02872', '0.02295', '-0.04295', '-0.08035', '-0.03366', '0.03433']], dtype=object)

In [10]:
accXList[1] # zeige Liste

['0.08288',
 '0.02032',
 '-0.06039',
 '-0.07898',
 '0.02872',
 '0.02295',
 '-0.04295',
 '-0.08035',
 '-0.03366',
 '0.03433']

In [11]:
np.array(accXList).dtype

dtype('<U9')

In [12]:
np.array(accXList).astype(float).dtype 

dtype('float64')

In [13]:
accXNp = np.array(accXList).astype(float) 
accXDf = pandas.DataFrame(accXNp)
accXDf.describe()

,0,1,2,3,4,5,6,7,8,9
count,4033.000000,4033.000000,4033.000000,4033.000000,4033.000000,4033.000000,4033.000000,4033.000000,4033.000000,4033.000000
mean,0.013317,-0.003369,0.007241,-0.014966,-0.016618,-0.013145,0.009173,0.030406,0.026682,-0.005150
std,1.441413,1.396812,1.419186,1.375713,1.404196,1.432436,1.451934,1.411490,1.394680,1.378802
min,-18.717620,-9.680360,-9.705260,-11.227190,-17.008060,-17.291380,-12.124190,-11.063110,-14.939860,-10.521570
25%,-0.619690,-0.667260,-0.637760,-0.651450,-0.661120,-0.629340,-0.574460,-0.605470,-0.597230,-0.632470
50%,0.006970,-0.012300,-0.007360,0.001700,-0.004460,-0.001930,0.014100,0.012560,0.008980,0.007730
75%,0.647860,0.613180,0.632160,0.646230,0.634240,0.646670,0.656000,0.672200,0.662620,0.628900
max,15.214150,15.588010,16.929340,10.079560,9.477870,9.770990,15.274030,16.080580,12.569330,11.386370


In [15]:
accXDf.shape

(4033, 10)

In [58]:
# Die Anzahl der Messerte muss immer gleich sein, ansonsten werden fehlende Messwerte
# mit NaN gefüllt.
featuresDf.Messwerte[featuresDf.Messwerte != 10]

Series([], Name: Messwerte, dtype: int64)

In [16]:
# Konvertierung von Object zu Float Werten in einer Zeile 
accYDf = pandas.DataFrame(np.array(featuresDf.AccelerometerY.str.split(" ").tolist()).astype(float))

In [17]:
accYDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4033 entries, 0 to 4032
Data columns (total 10 columns):
0    4033 non-null float64
1    4033 non-null float64
2    4033 non-null float64
3    4033 non-null float64
4    4033 non-null float64
5    4033 non-null float64
6    4033 non-null float64
7    4033 non-null float64
8    4033 non-null float64
9    4033 non-null float64
dtypes: float64(10)
memory usage: 315.2 KB


In [18]:
accYDf.head(2)

,0,1,2,3,4,5,6,7,8,9
0,0.05132,0.01027,0.02339,-0.01208,-0.00981,-0.02333,-0.03414,-0.02746,0.00854,0.00668
1,-0.05313,-0.02732,-0.02200,-0.03308,-0.02662,-0.00611,-0.00504,-0.08080,-0.14140,-0.03666


In [19]:
accZDf = pandas.DataFrame(np.array(featuresDf.AccelerometerZ.str.split(" ").tolist()).astype(float))

In [20]:
nickDf = pandas.DataFrame(np.array(featuresDf.Nick.str.split(" ").tolist()).astype(float))

In [21]:
rollDf = pandas.DataFrame(np.array(featuresDf.Roll.str.split(" ").tolist()).astype(float))

In [22]:
azimuthDf = pandas.DataFrame(np.array(featuresDf.Azimuth.str.split(" ").tolist()).astype(float))

In [23]:
accXMean = accXDf.T.mean()
accXMean.head(2)

0    0.009186
1   -0.010713
dtype: float64

In [24]:
# Neue Spalten werden erstellt und Mittelwerte zugewiesen
featuresDf['MittelX'] = accXMean
featuresDf['MittelY'] = accYDf.T.mean()
featuresDf['MittelZ'] = accZDf.T.mean()
featuresDf['MittelNick'] = nickDf.T.mean()
featuresDf['MittelRoll'] = rollDf.T.mean()

In [25]:
featuresDf.MittelX.head(2)

0    0.009186
1   -0.010713
Name: MittelX, dtype: float64

In [26]:
featuresDf.columns

Index(['Zeitstempel', 'Breitengrad', 'Laengengrad', 'Geschwindigkeit',
       'AccelerometerX', 'AccelerometerY', 'AccelerometerZ', 'Azimuth', 'Nick',
       'Roll', 'SensorZeitstempel', 'Messwerte', 'StartBewegungsD',
       'StartBelichtung', 'Belichtungszeit', 'MittelX', 'MittelY', 'MittelZ',
       'MittelNick', 'MittelRoll'],
      dtype='object')

Berechnet die Varianz. Dieser Funktion muss als Paramter der Mittelwert (mean) und der DataFrame mit   
den Float Werten übergeben werden, um die Varianz zu berechnen. Als Varianz wird der Durchschnitt der quadrierten  
Differenzen zum Mittelwert bezeichnet.   
Prec.:    
Postc.: Gibt die berechnete Variance als Float zurück oder 0 wenn die Anzahl der Werte <= 0  

In [27]:
def calcVariance(meansDf, dfValues):
    variance = []
    for i, it in dfValues.iterrows():
        sum = 0
        for value in it:
            tempDifference = value-meansDf[i]
            sum += tempDifference * tempDifference
        variance.append("{0:.5f}".format(round(sum / it.count(),5)))
    return np.array(variance).astype(float)

Hier wird die Standardabweichung der Varianz berechnet. Dies ist die Wurzel der Varianz.   
Für die Berechnung wird der Absolutwert von der Varianz genommen.    
Prec.:
Postc.:  Standardabweichung wird zurückgegeben.

In [28]:
def calculateStandardDeviation(varianceDf):
    deviation = []
    for v in varianceDf:
        wasNegative = False
        if(v < 0.0):
            wasNegative = True
        temp = math.sqrt(np.abs(v))
        if(wasNegative):
            temp *= -1
        deviation.append("{0:.5f}".format(round(temp,5)))
    return np.array(deviation).astype(float)

Die Funktion berechnet die Winkeldifferenz in rad zwischen den ersten gemessenen Gierwinkel und den letzten Gierwinkel   
innerhalb einer als Argument übergebenen DataFrames. Der zurückgegebene Radiant    
ist immer positiv und gibt die relative änderung in rad an.      
Prec.:   
Postc.: Radiant berechnet   

In [29]:
def calculateAngelChangeAzimuth(azimuthDf):
    azimuthL = []
    result = []
    counter = 0
    for i,values in azimuthDf.iterrows():
        resultTemp = 0
        for rad in values:
            azimuthL.append(rad * (180 / math.pi))
            counter = counter + 1
        resultTemp = np.abs(azimuthL[0] - azimuthL[counter-1])
        if(resultTemp > 180):
            resultTemp = 360 - resultTemp
        result.append("{0:.5f}".format(round(resultTemp * ( math.pi / 180),5)))
    return np.array(result).astype(float)

In [30]:
featuresDf['AzimuthAenderung'] = calculateAngelChangeAzimuth(azimuthDf)

In [31]:
# Berechne Varianz
featuresDf['VarianzX'] = calcVariance(featuresDf.MittelX,accXDf)
featuresDf['VarianzY'] = calcVariance(featuresDf.MittelY,accYDf)
featuresDf['VarianzZ'] = calcVariance(featuresDf.MittelZ,accZDf)
featuresDf['VarianzNick'] = calcVariance(featuresDf.MittelNick,nickDf)
featuresDf['VarianzRoll'] = calcVariance(featuresDf.MittelRoll,rollDf)

In [32]:
featuresDf.VarianzX.head()

0    0.00095
1    0.00280
2    0.00030
3    0.00177
4    0.00135
Name: VarianzX, dtype: float64

In [33]:
# Berechne Standardabweichung
featuresDf['AbweichungX'] = calculateStandardDeviation(featuresDf.VarianzX)
featuresDf['AbweichungY'] = calculateStandardDeviation(featuresDf.VarianzY)
featuresDf['AbweichungZ'] = calculateStandardDeviation(featuresDf.VarianzZ)
featuresDf['AbweichungNick'] = calculateStandardDeviation(featuresDf.VarianzNick)
featuresDf['AbweichungRoll'] = calculateStandardDeviation(featuresDf.VarianzRoll)

In [34]:
featuresDf.AbweichungX.head()

0    0.03082
1    0.05292
2    0.01732
3    0.04207
4    0.03674
Name: AbweichungX, dtype: float64

In [35]:
accXDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4033 entries, 0 to 4032
Data columns (total 10 columns):
0    4033 non-null float64
1    4033 non-null float64
2    4033 non-null float64
3    4033 non-null float64
4    4033 non-null float64
5    4033 non-null float64
6    4033 non-null float64
7    4033 non-null float64
8    4033 non-null float64
9    4033 non-null float64
dtypes: float64(10)
memory usage: 315.2 KB


In [36]:
accXDf.head()

,0,1,2,3,4,5,6,7,8,9
0,0.04156,0.00258,-0.01328,-0.04130,-0.03307,-0.01116,0.03702,0.04491,0.03590,0.02870
1,0.08288,0.02032,-0.06039,-0.07898,0.02872,0.02295,-0.04295,-0.08035,-0.03366,0.03433
2,-0.01467,-0.02708,-0.05233,-0.02657,-0.05193,-0.05689,-0.03022,-0.00887,-0.03777,-0.06088
3,-0.05236,-0.08788,-0.02436,-0.03483,0.00275,0.03282,0.04155,0.03322,0.04187,-0.01249
4,-0.10448,-0.05297,-0.04240,-0.01863,-0.01493,0.00337,0.00267,0.00211,0.00166,0.03194


In [37]:
featuresDf.MittelX.head()

0    0.009186
1   -0.010713
2   -0.036721
3   -0.005971
4   -0.019166
Name: MittelX, dtype: float64

In [38]:
# Änderungen in CSV Datei schreiben
featuresDf.set_index('Zeitstempel', inplace=True)
featuresDf.to_csv('merkmale.csv')

In [39]:
pandas.read_csv("merkmale.csv").columns

Index(['Zeitstempel', 'Breitengrad', 'Laengengrad', 'Geschwindigkeit',
       'AccelerometerX', 'AccelerometerY', 'AccelerometerZ', 'Azimuth', 'Nick',
       'Roll', 'SensorZeitstempel', 'Messwerte', 'StartBewegungsD',
       'StartBelichtung', 'Belichtungszeit', 'MittelX', 'MittelY', 'MittelZ',
       'MittelNick', 'MittelRoll', 'AzimuthAenderung', 'VarianzX', 'VarianzY',
       'VarianzZ', 'VarianzNick', 'VarianzRoll', 'AbweichungX', 'AbweichungY',
       'AbweichungZ', 'AbweichungNick', 'AbweichungRoll'],
      dtype='object')

In [37]:
# Unittests
import unittest

class CalcTest(unittest.TestCase):
    
    
    # Testet die Funktion calcVariance mit einem DataFrame mit den Werten [2.0, 2.0]
    # Der Mittelwert sollte 2.0 sein
    # Das erwartete Ergebnis ist 0.0
    def testCalcVariance2(self):
        meanL = [2.0]
        meanDf = pandas.DataFrame(meanL)
        valuesA = [2.0,2.0]
        valuesDf = pandas.DataFrame([[2.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf), 0.0,2)
        
    # Testet die Funktion calcVariance mit einem DataFrame mit den folgenden Werten:
    # [1.24, 2.5213, 10.434, 42.45, 5.9]
    # Der Mittelwert ist 12.508
    # Die erwartete Varianz ist 234,247016
    def testCalcVariance5(self):
        meanDf = pandas.DataFrame([12.508])
        valuesDf = pandas.DataFrame([[1.24, 2.52, 10.43, 42.45, 5.9]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf), 234.24702,5)   
    
    # Testet calcVariance mit 0 Datenwerten   
    def testCalcVariance0(self):
        meanDf = pandas.DataFrame([0.0])
        valuesDf = pandas.DataFrame([[0.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf),0.0,2)  
        
    # Testet calcVariance mit negativen Datenwerten   
    def testCalcVarianceNegative(self):
        meanDf = pandas.DataFrame([-24.0])
        valuesDf = pandas.DataFrame([[-2.0,-24.0,-5.0,7.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf),451.5,1) 
        
    # Testet calcVariance mit Muultidimensionalen DataFrame   
    def testCalcVarianceMultidim(self):
        meanDf = pandas.DataFrame([-24.0,0.0])
        valuesDf = pandas.DataFrame([[-2.0,-24.0,-5.0,7.0],[0.0,0.0,0.0,0.0]])
        # Hier wird Numpy Test verwendet um das Ergebnis (zwei Arrays) zu vergleichen
        npt.assert_array_equal(calcVariance(meanDf[0],valuesDf),np.array([ 451.5,0.0])) 
        
    # Dieser Test testet die Funktion calculateStandardDeviation.
    def testCalculateStandardDeviation(self):
        varianceDf = pandas.DataFrame([451.5])
        npt.assert_almost_equal(calculateStandardDeviation(varianceDf[0]),21.24853,5) 

    # Testet die Funktion calculateStandardDeviation.
    # Dabei ist das Argument negativ.
    def testCalculateStandardDeviationNegative(self):
        varianceDf = pandas.DataFrame([-451.5])
        npt.assert_almost_equal(calculateStandardDeviation(varianceDf[0]),-21.24853,5)
        
    # Testet die Funktion calculateStandardDeviation.
    # Dabei hat das Argument den Wert 0.0.
    def testCalculateStandardDeviationZero(self):
        varianceDf = pandas.DataFrame([0.0])
        npt.assert_almost_equal(calculateStandardDeviation(varianceDf[0]),0.0,2)
    
    #  Testet die Funktion calculateAngelChangeAzimuth mit zwei gleichen rad Werten
    def testCalculateAngelChangeAzimuthEqual(self):
        radiants = pandas.DataFrame([[1.0,1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),0.0,2)
    
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten -1.0 und 1.0
    def testCalculateAngelChangeAzimuthOne(self):
        radiants = pandas.DataFrame([[-1.0,1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),2.0,2)  
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten 1.0 und -1.0
    def testCalculateAngelChangeAzimuthOne2(self):
        radiants = pandas.DataFrame([[1.0,-1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),2.0,2)  

    # Testet die Funktion calculateAngelChangeAzimuthValues mit mehreren Werten
    def testCalculateAngelChangeAzimuthValues(self):
        radiants = pandas.DataFrame([[-1.57,1.0,2.0,1.3,1.57]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),3.14,2)
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten 0.05 und 0.06
    def testCalculateAngelChangeAzimuthSmall(self):
        radiants = pandas.DataFrame([[0.01,0.02]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants), 0.01,2)  

unittest.main(argv=[''], verbosity=2, exit=False)

testCalcVariance0 (__main__.CalcTest) ... ok
testCalcVariance2 (__main__.CalcTest) ... ok
testCalcVariance5 (__main__.CalcTest) ... ok
testCalcVarianceMultidim (__main__.CalcTest) ... ok
testCalcVarianceNegative (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthEqual (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthOne (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthOne2 (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthSmall (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthValues (__main__.CalcTest) ... ok
testCalculateStandardDeviation (__main__.CalcTest) ... ok
testCalculateStandardDeviationNegative (__main__.CalcTest) ... ok
testCalculateStandardDeviationZero (__main__.CalcTest) ... ok

----------------------------------------------------------------------
Ran 13 tests in 0.210s

OK
